In [ ]:
import pandas as pd
import numpy as np
import rdflib
import pykeen
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

## Loading knowledge graph

In [ ]:
reference_knowledge_graph = rdflib.Graph()
reference_knowledge_graph.parse("data/reference-kg.nt")

reference_knowledge_graph_trimmed = rdflib.Graph()
reference_knowledge_graph_trimmed.parse("data/reference-kg-1000.nt")

## Converting to pykeen format

In [ ]:
reference_data_numpy = np.array([(s, p, o) for s, p, o in reference_knowledge_graph])
reference_data_pykeen = TriplesFactory.from_labeled_triples(reference_data_numpy)
reference_data_pykeen.create_inverse_triples = True

## Learning the embeddings

In [ ]:
training, testing, validation = reference_data_pykeen.split([0.8, 0.1, 0.1])
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='TransE',
)

result.save_to_directory()